# Constellation and Chain Analysis: MultiConstellation Multihop MultiPath

<img src="MultipleConstellations.jpg" alt="Drawing" style="width: 500px;"/>

**Terminology**
* Node = Object in STK
* Edge = Access between two objects in STK
* Strand = The sequence of nodes and edges to complete access in a chain

**This notebook shows how to:**
* Find the shortest path between a starting and ending constellation, with many potential intermediate constellations.
* A chain will be built between each sequential pair in the constellationOrder list. Then networkx will be used build the network with the nodes coming from the constellations and the connections between the nodes coming from the chain accesses. Multiple sublists can be passed into constellationOrderList.
* Typical STK constraints such as range, link duration, Eb/No, etc are taken into account
* Data in the df variable can be pushed back into STK as a user supplied variable, a strand can be shown using object lines, and active objects over the analysis time period or at a time instance can be turned on.

**To reduce the runtime on subsequent runs, this scripts saves various computations to binary files and are reused for subsequent runs.** 
* The strands from the chains will be saved in the SavedNodes folder
* the nodes and associated time delays will be saved in the SavedNodes folder
* the node positions over time are saved in the SavedPositions folder
* the accesses between nodes over time are saved in the SavedEdges folder. 
* These folders will be created as subfolders of the directory used to run the script.
* During the first run the files will automatically be built and saved, subsequent runs will reload these files.
* To make changes simply delete the associated .pkl file for any changed strands, nodes, etc. and the script will recompute the data as needed. Or force all of the data to be overridden by setting the override options to be True.

In [ ]:
import numpy as np
import pandas as pd

pd.set_option("max_colwidth", 120)
from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.utilities.colors import Colors
from agi.stk12.stkobjects import *
from agi.stk12.stkutil import *
from agi.stk12.vgt import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from chainPathLib2 import *
import time
import networkx as nx

folders = [
    "SavedNodes",
    "SavedPositions",
    "SavedStrands",
    "SavedEdges",
    "SavedNetworkData",
]
for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)
import warnings

warnings.filterwarnings("ignore")  # Ignore some numpy indexing warnings

## Constellation Connection Order, Computation Time, Save Data Options

Often the hardest part of network analysis is defining the constellation order. Here are some tips:

* Every pair of constellations in the constellationOrderLists will create a one way flow of information (a directed edge) .i.e. [constellation1,constellation2] means constellation1 -> constellation2 but **NOT** constellation2 -> constellation1
    
* To allow for multiple hops between constellations the constelltionOrderLists needs to create a cycle:
    * If you want a constellation to link to itself simply order [constellation1,constellation1]
    * If you want to link to another constellation before linking back to the original [constellation1,constellation2,constellation1]
    * Theoreitcally this cycle can be as big as you want. [c1,c2,....,cN,c1]
    * The reason adding cycles of constellations works is because constellations have multiple objects, so the connections will always be to different objects in the constellation therefore you won't revisit specific nodes in the final path. If the constellation at the beginning of the cycle has only 1 object the cycle will be ignored
    * <img src="CycleIllustration.jpg" alt="Drawing" style="width: 500px;"/>
* The constellationOrderLists **does NOT** require the final path to include every constellation in order, it will find the shortest path among possible connections and cut out unneccesary looping. For example, say you define consetllationOrderLists to be [[start,satConstellation,satConstellation,end]] because you want to allow for intersatellite links in satConstellation, the shortest path may just be from start->satConstellation->end without the need for intersatellitelinks, in which case the intersatellite linking will not be used.
    * This code does not force the use of a cycle aka force intersatellite links to be used in the above example. If you want to do this you have 2 options. Use the prebuilt chains notebook (but it doesn't support multiple hops) or compute multiple shortest paths and filter out any which don't have the desired properties. In the future an improvement may be made to add additional constraints on the shortest path.
    
* To constrain the path for children objects (sensors, transmitters, recievers) you may need to add logical Parent "From" and "To" constraints on the constellations in STK. Think of "From" as the 1st position in a connection (aka transmitting) and "To" as the 2nd position in a connection (aka reiceving). Often you want transmitters to have a "From" Different Parents constraint and recievers to have a "From" Same Parents constraint. The constellation ordering can be set up in a way to force the paths to contain the parent object or to bypass it and only use the children objects. i.e. only use transmitters and recievers and not go to a satellite bus first. For example either of the following orderings could be valid [recievers,satellites,transmitters] (the satellites constellation would need a "From" Same Parent constraint) or [transmitters,receivers].

In [ ]:
# The constellationOrderLists builds a directed chain for each adjacent pair in the list
# i.e.: constellationOrderLists = [[A,B],[X,Y,Z,X]] would build chainAB,chainXY,chainYZ,chainZX
constellationOrderLists = [
    [
        "Targets",
        "ObservingSatsFORs",
        "ObservingSats",
        "ObservingSatsTransmitters",
        "ObservingSatsReceivers",
        "ObservingSats",
        "RelaySatsFORs",
        "RelaySats",
        "RelaySatsFORs",
        "EndLocations",
    ],
    ["RelaySats", "RelaySats"],
]

startingConstellation = "Targets"  # complete path starting constellation
endingConstellation = "EndLocations"  # complete path ending constellation

start = 0  # EpSec
stop = 120 * 60  # EpSec
step = 10  # sec # Time resolution of computations
nodeDelays = {
    "ObservingSatsFORs": 0.01,
    "ObservingSatsTransmitters": 0.005,
    "ObservingSatsReceivers": 0.005,
    "RelaySatsFORs": 0.01,
    "RelaySats": 0.002,
}  # Add in time delays. Provide the constellation name in STK and the node delays

stkVersion = 12
overrideStrands = False  # Override previously computed chains
overrideNodeDelaysByNode = False  # Override previously built node delay dictionaries
overrideNodesTimesPos = False  # Override previously built node positions

In [ ]:
# Connect to STK
stkApp = STKDesktop.AttachToApplication()
stkRoot = stkApp.Root
stkRoot.Isolate()
stkRoot.UnitPreferences.SetCurrentUnit(
    "DateFormat", "EpSec"
)  # Units to EpSec for ease of use
stkRoot.ExecuteCommand('Units_SetConnect / Date "EpochSeconds"')
try:
    stkRoot.ExecuteCommand("VO * ObjectLine DeleteAll")
    # Clean up old object lines
except:
    pass

# Build chains and create a dict of time delays for each node
t1 = time.time()
chainNames = createDirectedChains(
    stkRoot,
    constellationOrderLists,
    start=start,
    stop=stop,
    color=Colors.FromRGB(0, 196, 196),
)
print(time.time() - t1)
t1 = time.time()
nodeDelaysByNode = getNodeDelaysByNode(
    stkRoot, nodeDelays, chainNames=chainNames, overrideData=overrideNodeDelaysByNode
)
print(time.time() - t1)

startingNodes = getNodesFromConstellation(stkRoot, startingConstellation)
endingNodes = getNodesFromConstellation(stkRoot, endingConstellation)

## Compute Strands, Strands at Each Time, Node Positions, Edges, Distances, Time Delays

Only need to run this section once unless you change the section above, i.e. the objects in the constellations,the constellation ordering, the start, stop and step times, nodeDelays


In [ ]:
# Compute strands, (this is actually a list of all intervals for each edge)
t1 = time.time()
# strands,dfStrands = getAllStrands(stkRoot,chainNames,start,stop,overrideData=overrideStrands)
strands = getAllStrands(stkRoot, chainNames, start, stop, overrideData=overrideStrands)[
    0
]  # if memory or performance is an issue don't output dfStrands
print(time.time() - t1)
# dfStrands

In [ ]:
# Compute node positions, distances and time delays

t1 = time.time()
strandsAtTimes = getStrandsAtTimes(
    strands, start, stop, step
)  # Discretize strand intervals into times
t2 = time.time()
print(t2 - t1)

t1 = time.time()
nodesTimesPos = computeNodesPosOverTime(
    stkRoot, strands, start, stop, step, overrideData=overrideNodesTimesPos
)  # Pull node position over time
t2 = time.time()
print(t2 - t1)

# Check if data needed for strandsAtTimes is missing, force a recomputed if needed
t1 = time.time()
strands, nodesTimesPos, strandsAtTimes = recomputeMissingData(
    stkRoot,
    strands,
    start,
    stop,
    step,
    chainNames,
    nodesTimesPos,
    strandsAtTimes,
    recomputeIfDataIsMissing=True,
)
t2 = time.time()
print(t2 - t1)

t1 = time.time()
timeNodePos = computeTimeNodePos(
    strandsAtTimes, nodesTimesPos
)  # Nodes and positions at each time
t2 = time.time()
print(t2 - t1)

t1 = time.time()
timesEdgesDistancesDelays = computeTimeEdgesDistancesDelays(
    strandsAtTimes, nodesTimesPos, nodeDelaysByNode, overrideData=True
)  # Edges, distances and delays at each time
t2 = time.time()
print(t2 - t1)

## Use NX for Network Metrics and Reliability Analysis

This notebook breaks the starting and ending nodes into unique pairs and finds the shortest path between each pair. It also supports finding the TopN best paths for each unique pair.

You can change the filename to save different runs

In [ ]:
# Get pairs of each starting and ending node permutation in the constellations
nodePairs = [
    (start, end) for start, end in itertools.product(startingNodes, endingNodes)
]  # full permutation
pd.DataFrame(nodePairs, columns=["start", "end"])

In [ ]:
# Loop through each node pair and compute network metrics
# Edit computeNetworkMetrics in chainPathLibCustom to for additional metrics
overrideNetwork = False
metric = "distance"  # 'distance' or 'timeDelay'
filename = ""  # a file for each nodepair will created and the names of the node be appended to filename, leave empty if desired

for nodePair in nodePairs:
    df = computeNetworkMetrics(
        start,
        stop,
        step,
        timeNodePos,
        timesEdgesDistancesDelays,
        [nodePair[0]],
        [nodePair[1]],
        metric,
        computeNumNodesToLoseAccessBetweenAnyPair=False,
        overrideData=overrideNetwork,
        printTime=True,
        diNetwork=True,
        filename=filename,
    )

In [ ]:
df

##  Investigate Routing between Specific Nodes

In [ ]:
# Pick a starting and ending node
startingNode = "Target/Target7"
endingNode = "Place/Washington_DC"

In [ ]:
# Load df
filenameToLoad = "SavedNetworkData/df{}{}{}.pkl".format(
    filename, startingNode.split("/")[-1], endingNode.split("/")[-1]
)
with open(filenameToLoad, "rb") as f:
    df = pickle.load(f)
df = addLightAndNodeDelays(df, timesEdgesDistancesDelays)
dfIntervals = createDfIntervals(df, stop, step)
addStrandsAsObjectLines(stkRoot, dfIntervals, color="yellow")

In [ ]:
# Add data back into STK for reporting and plotting
t1 = time.time()
df["distance"] = df["distance"] * 1000  # May need to fix meter/kilometer issue
addDataToSTK(
    stkRoot, chainNames[0], df
)  # Adds data in df back into STK to the first chain under User Supplied data
print(time.time() - t1)
df

In [ ]:
dfIntervals["strand"][0]

In [ ]:
# Active objects in the network over time
objPaths = list(set((item for sublist in df["strand"] for item in sublist)))

In [ ]:
# Turn on the objects in the scenario
turnGraphicsOnOff(stkRoot, objPaths, onOrOff="On", parentsOnly=True)

In [ ]:
# Turn off the objects in the sceario
turnGraphicsOnOff(stkRoot, objPaths, onOrOff="Off", parentsOnly=True)

## Investigate Instances in Time

In [ ]:
# Look at an instance in time (pick a time in df)
t = 0
stkRoot.CurrentTime = t

In [ ]:
# Look at strand order and the node delay
objPaths = df["strand"][t / step]
nodeDelaysByStrand = {node: nodeDelaysByNode[node] for node in objPaths}
pd.DataFrame([*nodeDelaysByStrand.items()], columns=["node", "nodeDelay"])

In [ ]:
# look at possible connections for the object of interest at that time
nodeInterest = objPaths[0]
possibleNodeConnections(t, nodeInterest, timesEdgesDistancesDelays)

In [ ]:
# Turn on the objects in the scenario
turnGraphicsOnOff(stkRoot, objPaths, onOrOff="On", parentsOnly=False)

In [ ]:
# Turn off the objects in the sceario
turnGraphicsOnOff(stkRoot, objPaths, onOrOff="Off", parentsOnly=False)

## Investigate Node Utilization

In [ ]:
# Most frequnt node in the shortest path and the sum of their durations
strands = dfIntervals[["strand", "start", "stop"]].values
dfNodesIntervals = getNodesIntervalsFromStrands(strands)
dfNodeActive = getActiveDuration(dfNodesIntervals, start, stop)
dfNodeActive.sort_values("sum dur", ascending=False).head(10)

## Compute the Top N Paths for Multiple Node Pairs

In [ ]:
# Loop through each node pair and compute network metrics for top N strands, assumes only 1 starting and ending node for each network
# The strands will all be unique, but large portions of the strand may be reused unless the nodes are edges are removed
topN = 5
removeUsedNodes = False  # Remove any nodes in previous shortest paths
removeUsedEdges = False  # Remove any edge in previous shortest paths
for nodePair in nodePairs:
    startingNode = nodePair[0]
    endingNode = nodePair[1]
    dfTopN = computeNetworkTopN(
        start,
        stop,
        step,
        timeNodePos,
        timesEdgesDistancesDelays,
        startingNode,
        endingNode,
        metric,
        topN=topN,
        overrideData=overrideNetwork,
        printTime=True,
        removeUsedNodes=removeUsedNodes,
        removeUsedEdges=removeUsedEdges,
        filename=filename,
    )

## Load And Merge Multiple Networks

In [ ]:
nodePairsToLoad = [
    ("Target/Target1", "Place/Los_Angeles_CA"),
    ("Target/Target1", "Place/Washington_DC"),
    ("Target/Target2", "Place/Los_Angeles_CA"),
    ("Target/Target2", "Place/Washington_DC"),
]  # Can specifically give pairs of interest
nodePairsToLoad = nodePairs  # Use all of the nodePairs
pd.DataFrame(nodePairsToLoad, columns=["node1", "node2"])

In [ ]:
# Only load needed times
neededTimes = np.array(list(timesEdgesDistancesDelays.keys()))

# # Load multiple networks into one using TopN
# df = loadNetworkDfTopN(nodePairsToLoad,topN,neededTimes=neededTimes)
# df = addLightAndNodeDelays(df,timesEdgesDistancesDelays)
# df

# Load multiple networks into one
df = loadNetworkDf(nodePairsToLoad, neededTimes=neededTimes)
df = addLightAndNodeDelays(df, timesEdgesDistancesDelays)
df

In [ ]:
# Load the paths into STK, pick from a color map or specify a specifc color

# Create intervals for each edge and count the number of times the edge is active
# Optionally specify an edge weight which will be multipled by its frequency at each time step
timesEdgeCountAll = createTimesEdgesCountFromDF(df)

# # Consider binning the counts to prevent a ton of unique colors, which can noteably speed up the time it takes to add data to STK
# # Look at the pd.cut documentation for specifying specific bin edges
# numBins = 5
# bins = pd.cut(timesEdgeCountAll[:,2],numBins,labels=False)
# timesEdgeCountAll[:,2] = np.round(bins*(max(timesEdgeCountAll[:,2])-min(timesEdgeCountAll[:,2]))/numBins+min(timesEdgeCountAll[:,2]),6)

t1 = time.time()
# Color Maps here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
# Good options: diverging and sequential color maps such as: cool,coolwarm,Wistia
cmap = cm.get_cmap(
    "coolwarm"
)  # pick a matplotlib color map or don't pass in a colorMap to just use a color
addTimesEdgesCountAsObjectLines(
    stkRoot,
    timesEdgeCountAll,
    step,
    addTo2D=False,
    color="%000196196",
    colorMap=cmap,
    lineWidth=7,
)
print(time.time() - t1)

if cmap:
    plotColorbar(timesEdgeCountAll, cmap, plotBoth=True, tickRotationInDeg=65)